# Data-Efficient Hierarchical Reinforcement Learning

## Parameterized Rewards

* 考虑Fig. 2中，从$g_0$到$g_{c-1}$，然后有说goal transition为：

  \begin{equation*}
  h(s_t,g_t,s_{t+1})=s_t+g_t-s_{t+1}
  \end{equation*}

  从这个角度，难道$g_{t+1}=h(s_t,g_t,s_{t+1})$，TODO

  若成立，则会有：

  \begin{align*}
  g_{t+1}&=s_t+g_t-s_{t+1}\\
  g_{t+2}&=s_{t+1}+g_{t+1}-s_{t+2}
  \end{align*}

  所以，telescope之后，会得到：$g_{t+2}=s_t+g_t-s_{t+2}$

* 另一方面，从$r(s_t,g_t,a_t,s_{t+1})=-\|s_t+g_t-s_{t+1}\|_2$，所以，其是希望$\|g_{t+1}\|_2\to 0$，TODO

* That is, at step $t$, the higher-level policy produces a goal $g_t$, indicating its desire for the lower-level
agent to take actions that yield it an observation $s_{t+c}$ that is close to $s_t + g_t$.

  如果$g_{t+c}=s_t+g_t-s_{t+c}$，那么$\|g_{t+c}\|_2\to 0$就实现了close to $s_t + g_t$的目标，TODO

* DDPG的方式，针对lower-level policy
    * 原本的形式：
        * average Bellman error

          \begin{equation*}
          \bigl(Q_\theta(s_t,a_t)-R_t-\gamma Q_\theta(s_{t+1},\mu_\phi(s_{t+1})\bigr)^2
          \end{equation*}
        
        * 而$\mu_\phi$需maximize $Q_\theta(s_t,\mu_\phi(s_t))$
    * 新的形式：
        * simply incorporating gt as an
additional input into the value and policy models
        * $Q^{lo}_\theta(s_t,g_t,a_t)$
        
        * policy

          \begin{equation*}
          \mu^{lo}_\phi(s_{t+1},g_{t+1})
          \end{equation*}

## Off-Policy Corrections for Higher-Level Training

* lower-level policy所拥有的信息：

  \begin{equation*}
  (s_{t:t+c-1},g_{t:t+c-1},a_{t:t+c-1},R_{t:t+c-1},s_{t+c})
  \end{equation*}

  需要将这些转换为higher-level policy能用的信息：

  \begin{equation*}
  (s_t,g_t,\sum R_{t:t+c-1},s_{t+c})
  \end{equation*}

  注意$x_{t:t+c-1}$只是表示sequence，所以需要$\sum$

  $g_t$是higher-level policy的action

* goal transition时用的$h$是fixed，即使是在correction的环节，所以，$g_{t:t+c-1}$中只有$g_t$是可以操作的

* 该环节是对$g_t$进行re-label，得到$\tilde{g}_t$

* 关注的是

  \begin{equation*}
  \mu^{lo}(a_{t:t+c-1}|s_{t:t+c-1},\tilde{g}_{t:t+c-1})
  \end{equation*}

  如何去maximize，可以操作的实际仅为$\tilde{g}_t$

  * 其的做法：
    * $\arg\max$上述的目标，相当于如下的目标
      
      \begin{equation*}
      -\frac{1}{2}\sum_{i=t}^{t+c-1}\|a_i-\mu^{lo}(s_i;\tilde{g}_i)\|^2_2
      \end{equation*}
    
    * 如何approximately solve

      10个candidates